<a href="https://colab.research.google.com/github/varsha6319/Product_Rec/blob/main/Step_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyspark==3.3.0

In [2]:
from pyspark.sql import SparkSession
import zipfile
import os

In [4]:
# # Download the RAPIDS Accelerator jar
# !wget https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/23.02.0/rapids-4-spark_2.12-23.02.0.jar

# # Download the cuDF jar
# !wget https://repo1.maven.org/maven2/ai/rapids/cudf/23.02.0/cudf-23.02.0-cuda11.jar


In [3]:
spark = SparkSession.builder \
    .appName("RandomForest_GPU") \
    .config("spark.executor.memory", "64g") \
    .config("spark.executor.cores", "16") \
    .config("spark.driver.memory", "64g") \
    .config("spark.ui.port", "4040").getOrCreate()

In [4]:
parquet_path1 = '/content/drive/My Drive/santander-product-recommendation/train_data.parquet'
parquet_path2 = '/content/drive/My Drive/santander-product-recommendation/transformed_intermediate_data_0528.parquet'
parquet_path3 = '/content/drive/My Drive/santander-product-recommendation/transformed_test_data.parquet'
parquet_path4 = '/content/drive/My Drive/santander-product-recommendation/actual_intermediate_data_0528_processed.parquet'

# Read Parquet files into Spark DataFrames
train_data = spark.read.parquet(parquet_path1)
transformed_intermediate_data_0528 = spark.read.parquet(parquet_path2)
original_intermediate_data_0528 = spark.read.parquet(parquet_path4)
test_data = spark.read.parquet(parquet_path3)

In [5]:
train_data = train_data.select("customer_code","features","multi_label","label")
transformed_intermediate_data_0528 = transformed_intermediate_data_0528.select("customer_code","features")
test_data = test_data.select("customer_code","features")

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col, when
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

In [7]:
# Define Random Forest classifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42, numTrees=10, maxDepth=5)  # Adjust parameters

In [8]:
# 1. Train the model on historical data up to 2016-04-28
trained_model = rf.fit(train_data)

In [14]:
# Predict products for 2016-05-28 using the trained model
predictions_0528 = trained_model.transform(transformed_intermediate_data_0528)

predicted_products_0528 = predictions_0528.select("customer_code", "prediction").withColumnRenamed("prediction", "predicted_products_0528")

# Combine predictions for 2016-05-28 with final data for 2016-06-28
combined_data_0628 = test_data.join(predicted_products_0528, on='customer_code', how='left')

# Predict new products for 2016-06-28 using the combined data
predictions_0628 = trained_model.transform(combined_data_0628)

In [16]:
predictions_0628.columns

['customer_code',
 'features',
 'predicted_products_0528',
 'rawPrediction',
 'probability',
 'prediction']

In [19]:
new_products_df = predictions_0628.withColumn("new_products", col("prediction") - col("predicted_products_0528")) \
                                  .select("customer_code", "new_products")

['customer_code', 'new_products']

In [9]:
# 2. Predict products for 2016-05-28 using the trained model
predictions_0528 = trained_model.transform(transformed_intermediate_data_0528)
predictions_0528 = predictions_0528.withColumnRenamed("features", "features_inter")
# 3. Combine predictions with final data for 2016-06-28
combined_data_0628 = test_data.join(predictions_0528, on='customer_code', how='left') \
                              .withColumnRenamed("prediction", "prediction_0528") \
                              .withColumnRenamed("rawPrediction", "rawPrediction_0528") \
                              .withColumnRenamed("probability", "probability_0528")

In [10]:
predictions_0628 = trained_model.transform(combined_data_0628)

In [11]:
new_products_0628 = (predictions_0628
                     .withColumn("new_products", col("prediction") - col("prediction_0528"))
                     .select("customer_code", "new_products"))

# new_products_0628.show()


In [28]:
new_products_0628.columns

['customer_code', 'new_products']

In [30]:
from pyspark.sql.functions import col, expr, array, when

# Define your product columns
product_columns = [
    "savings_account", "guarantees", "current_accounts", "derivada_account", "payroll_account",
    "junior_account", "mas_particular_account", "particular_account", "particular_plus_account",
    "short_term_deposits", "medium_term_deposits", "long_term_deposits", "e_account", "funds",
    "mortgage", "loans", "taxes", "credit_card", "securities", "home_account", "payroll",
    "pensions_nom", "pension_plans", "direct_debit"
]

# Define an expression to map prediction index to product column name
mapped_predictions_df = new_products_df.withColumn("predicted_product",
    expr(
        "CASE " +
        " ".join([
            f"WHEN new_products = {idx} THEN '{product_columns[idx]}' "
            for idx in range(len(product_columns))
        ]) +
        " ELSE 'unknown' END"
    )
)

In [35]:

columns = ['customer_code', 'new_products', 'predicted_product']

In [ ]:
mapped_predictions_df.select("customer_code","predicted_product").show()

In [42]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

parquet_path = '/content/drive/My Drive/santander-product-recommendation/predictions.parquet'

# Function to save DataFrame to Parquet with overwrite mode and error handling
def save_dataframe(df, path):
    try:
        df.write.mode('overwrite').parquet(path)
        print(f"DataFrame saved to {path}")
    except AnalysisException as e:
        print(f"Failed to save DataFrame to {path}. Error: {e}")


In [49]:
df = mapped_predictions_df.repartition(100)  # Adjust partition count as needed
df.write.csv('/content/drive/My Drive/santander-product-recommendation/output.csv', header=True, mode='overwrite')


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [48]:
mapped_predictions_df.write.csv("/content/drive/My Drive/santander-product-recommendation/output.csv", header=True, mode="overwrite")

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [33]:
from pyspark.sql.functions import count

mapped_predictions_df.groupBy("predicted_product").agg(count("*").alias("count")).show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from pyspark.sql import functions as F

# Assuming predictions_0528 is your DataFrame containing predictions and original_intermediate_data_0528 contains actual purchases
joined_data = predictions_0528.join(original_intermediate_data_0528, on='customer_code', how='inner')

# Initialize dictionaries to store counts
tp_counts = {}
fp_counts = {}
fn_counts = {}
tn_counts = {}

# Calculate TP, FP, FN, TN for each product column
for col_name in product_columns:
    tp_col = F.sum(F.when((F.col("prediction") == 1) & (F.col(col_name) == 1), 1).otherwise(0)).alias(f"tp_{col_name}")
    fp_col = F.sum(F.when((F.col("prediction") == 1) & (F.col(col_name) == 0), 1).otherwise(0)).alias(f"fp_{col_name}")
    fn_col = F.sum(F.when((F.col("prediction") == 0) & (F.col(col_name) == 1), 1).otherwise(0)).alias(f"fn_{col_name}")
    tn_col = F.sum(F.when((F.col("prediction") == 0) & (F.col(col_name) == 0), 1).otherwise(0)).alias(f"tn_{col_name}")

    counts = joined_data.select(tp_col, fp_col, fn_col, tn_col).first()

    tp_counts[col_name] = counts[f"tp_{col_name}"]
    fp_counts[col_name] = counts[f"fp_{col_name}"]
    fn_counts[col_name] = counts[f"fn_{col_name}"]
    tn_counts[col_name] = counts[f"tn_{col_name}"]

# Calculate precision, recall, and F1-score for each product column
accuracy_metrics = {}
for col_name in product_columns:
    tp = tp_counts[col_name]
    fp = fp_counts[col_name]
    fn = fn_counts[col_name]
    tn = tn_counts[col_name]

    precision = tp / (tp + fp + 1e-10)  # Adding small epsilon to avoid division by zero
    recall = tp / (tp + fn + 1e-10)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-10)

    accuracy_metrics[col_name] = {
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

# Print accuracy metrics for each product column
for col_name, metrics in accuracy_metrics.items():
    print(f"Product: {col_name}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1-score: {metrics['f1_score']:.4f}")
    print()


In [22]:
# product_columns = [
#     "savings_account", "guarantees", "current_accounts", "derivada_account", "payroll_account",
#     "junior_account", "mas_particular_account", "particular_account", "particular_plus_account",
#     "short_term_deposits", "medium_term_deposits", "long_term_deposits", "e_account", "funds",
#     "mortgage", "loans", "taxes", "credit_card", "securities", "home_account", "payroll",
#     "pensions_nom", "pension_plans", "direct_debit"
# ]
# from pyspark.sql.functions import array, col, expr, lit

# index_to_column = dict(enumerate(product_columns))
# def map_prediction_to_column(row):
#     customer_code = row.customer_code
#     prediction_index = int(row.prediction)  # Convert prediction to integer index
#     column_name = index_to_column.get(prediction_index, "unknown")  # Get corresponding column name
#     return (customer_code, column_name)


In [23]:
# # Assuming predictions is your DataFrame containing customer_code and prediction columns
# mapped_predictions_rdd = new_products_df.rdd.map(map_prediction_to_column)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Convert RDD back to DataFrame with appropriate schema
mapped_predictions_df = spark.createDataFrame(mapped_predictions_rdd, ["customer_code", "new_products"])

In [ ]:
# Show mapped predictions DataFrame
mapped_predictions_df.show()

In [ ]:
from pyspark.sql.functions import count

mapped_predictions_df.groupBy("new_products").agg(count("*").alias("count")).show()

In [45]:
# Assuming new_products_0628 is your DataFrame containing customer_code and new_products columns
mapped_new_products_0628 = map_new_products_to_names(new_products_0628, product_columns)


ParseException: 
Syntax error at or near 'WHEN': extra input 'WHEN'(line 1, pos 36)

== SQL ==
TRANSFORM(new_products, idx -> CASE WHEN idx > 0 THEN 'savings_account', 'guarantees', 'current_accounts', 'derivada_account', 'payroll_account', 'junior_account', 'mas_particular_account', 'particular_account', 'particular_plus_account', 'short_term_deposits', 'medium_term_deposits', 'long_term_deposits', 'e_account', 'funds', 'mortgage', 'loans', 'taxes', 'credit_card', 'securities', 'home_account', 'payroll', 'pensions_nom', 'pension_plans', 'direct_debit'[idx] ELSE NULL END)
------------------------------------^^^


In [ ]:
# Show the mapped DataFrame
mapped_new_products_0628.show()

In [ ]:
predictions_0528.join(original_intermediate_data_0528)

In [ ]:
# Show new products for 2016-06-28
new_products_0628.show()

In [12]:
trained_model = rf.fit(train_data)

In [16]:
# Predict products for 2016-05-28 using the trained model
predictions_0528 = trained_model.transform(transformed_intermediate_data_0528)

predicted_products_0528 = predictions_0528.select("customer_code", "prediction").withColumnRenamed("prediction", "predicted_products_0528")

# Combine predictions for 2016-05-28 with final data for 2016-06-28
combined_data_0628 = test_data.join(predicted_products_0528, on='customer_code', how='left')

# Predict new products for 2016-06-28 using the combined data
predictions_0628 = trained_model.transform(combined_data_0628)

In [26]:
predicted_products_0528.show()

+-------------+-----------------------+
|customer_code|predicted_products_0528|
+-------------+-----------------------+
|       538413|                    0.0|
|       538409|                    0.0|
|       538408|                    0.0|
|       538431|                    0.0|
|       538407|                    0.0|
|       538402|                    0.0|
|       538401|                    1.0|
|       538398|                    0.0|
|       538397|                    1.0|
|       538396|                    1.0|
|       538391|                    0.0|
|       538390|                    0.0|
|       538389|                    0.0|
|       538387|                    0.0|
|       538380|                    0.0|
|       538377|                    1.0|
|       538403|                    1.0|
|       538434|                    1.0|
|       538441|                    0.0|
|       538445|                    0.0|
+-------------+-----------------------+
only showing top 20 rows



In [25]:
transformed_intermediate_data_0528.show()

+-------------+--------------------+
|customer_code|            features|
+-------------+--------------------+
|       538413|(460,[0,5,124,125...|
|       538409|(460,[0,5,124,125...|
|       538408|(460,[0,5,124,125...|
|       538431|(460,[0,5,124,125...|
|       538407|(460,[0,5,123,125...|
|       538402|(460,[0,5,123,125...|
|       538401|(460,[0,5,123,125...|
|       538398|(460,[0,5,124,125...|
|       538397|(460,[0,5,124,125...|
|       538396|(460,[0,5,124,125...|
|       538391|(460,[0,5,123,125...|
|       538390|(460,[0,5,123,125...|
|       538389|(460,[0,5,123,125...|
|       538387|(460,[0,5,124,125...|
|       538380|(460,[0,5,123,125...|
|       538377|(460,[0,5,123,125...|
|       538403|(460,[0,5,124,125...|
|       538434|(460,[0,5,123,125...|
|       538441|(460,[0,5,123,125...|
|       538445|(460,[0,5,123,125...|
+-------------+--------------------+
only showing top 20 rows



In [17]:
# Combine predictions for 2016-05-28 with final data for 2016-06-28
combined_data_0628 = test_data.join(predicted_products_0528, on='customer_code', how='left')

# Predict new products for 2016-06-28 using the combined data
predictions_0628 = trained_model.transform(combined_data_0628)


In [19]:
new_products_0628 = (predictions_0628
                     .withColumn("new_products", col("prediction") - col("predicted_products_0528"))
                     .select("customer_code", "new_products"))


In [20]:
new_products_0628.show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [24]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Step 1: Split the Historical Data
train_data1, validation_data = train_data.randomSplit([0.8, 0.2], seed=42)

# Step 2: Train the Random Forest Model on Training Data
rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=10, maxDepth=10)
trained_model = rf.fit(train_data1)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Step 3: Make Predictions on Validation Data
validation_predictions = trained_model.transform(validation_data)

In [ ]:
# Step 4: Evaluate the Model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(validation_predictions)
print(f"Accuracy on validation data = {accuracy}")

# Calculate additional metrics if needed
precision_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

precision = precision_evaluator.evaluate(validation_predictions)
recall = recall_evaluator.evaluate(validation_predictions)
f1_score = f1_evaluator.evaluate(validation_predictions)

print(f"Precision on validation data = {precision}")
print(f"Recall on validation data = {recall}")
print(f"F1 Score on validation data = {f1_score}")

In [13]:
predictions_0528 = trained_model.transform(transformed_intermediate_data_0528)

In [14]:
predicted_products_0528 = predictions_0528.select("customer_code", "prediction").withColumnRenamed("prediction", "predicted_products_0528")


In [ ]:
# Combine predictions for 2016-05-28 with final data for 2016-06-28
combined_data_0628 = transformed_final_data_0628.join(predicted_products_0528, on='customer_code', how='left')

# Predict new products for 2016-06-28 using the combined data
predictions_0628 = trained_model.transform(combined_data_0628)

# Identify new products for 2016-06-28 that were not predicted for 2016-05-28
# Assuming 'prediction' column contains the latest predictions for 2016-06-28
new_products_0628 = (predictions_0628
                     .withColumn("new_products", col("prediction") - col("predicted_products_0528"))
                     .select("customer_code", "new_products"))

# Show new products for 2016-06-28
new_products_0628.show()

In [ ]:

# Transform the final data for 2016-06-28
transformed_final_data_0628 = fitted_pipeline.transform(final_data_0628)


In [ ]:
train_data = train_data.select("features","multi_label","label","date", "customer_code", "sex", "age", "country_residence", "new_customer_index", "customer_seniority", "customer_activity_index", "customer_type_start_month", "customer_relation_type_start_month", "residence_index", "foreigner_index",  "customer_segment", "household_income", "age_group", "activity_index_weighted", "savings_account", "guarantees", "current_accounts", "derivada_account", "payroll_account", "junior_account", "mas_particular_account", "particular_account", "particular_plus_account", "short_term_deposits", "medium_term_deposits", "long_term_deposits", "e_account", "funds", "mortgage", "loans", "taxes", "credit_card", "securities", "home_account", "payroll", "pensions_nom", "pension_plans", "direct_debit")

test_data = test_data.select("features", "date", "customer_code", "sex", "age", "country_residence", "new_customer_index", "customer_seniority", "customer_activity_index", "customer_type_start_month", "customer_relation_type_start_month", "residence_index", "foreigner_index","customer_segment", "household_income", "age_group", "activity_index_weighted")

In [ ]:
# Repartition for parallelism
train_data = train_data.repartition(100)

# Cache dataframe for iterative operations
train_data.cache()

In [11]:
from pyspark.sql.functions import col, when

In [ ]:
# Filter to include rows where date is between "2016-01-28" and "2016-05-28"

# train_data = train_data.filter((col("date") >= "2016-04-28") & (col("date") <= "2016-05-28"))

# Show the filtered DataFrame
# train_data.show()

In [10]:
train_data = train_data.select("customer_code","features","multi_label","label")
test_data = test_data.select("customer_code","features")

In [ ]:
# Show schema and some data to confirm successful read
# train_data.printSchema()
# train_data.show()

In [ ]:
# test_data.printSchema()
# test_data.show()

In [13]:
# Define Random Forest classifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42, numTrees=5, maxDepth=5)  # Adjust parameters

In [14]:
# Fit pipeline on training data
model = rf.fit(train_data)

In [ ]:
# model_rf2 - model that is imp

In [ ]:
# Define the local path where you want to save the model
model_local_path = "/content/model_rf2"

# Save the model to the local path
model.save(model_local_path)

import shutil

# Define the destination directory in Google Drive
drive_destination = '/content/drive/My Drive/santander-product-recommendation/'

# Copy the model directory to Google Drive
shutil.copytree(model_local_path, drive_destination + 'model_rf2')

In [15]:
# Make predictions on the transformed test data
predictions = model.transform(test_data)

# Show predictions
predictions.select("customer_code", "prediction").show()

+-------------+----------+
|customer_code|prediction|
+-------------+----------+
|       376061|       1.0|
|       376063|       0.0|
|       376065|       0.0|
|       376066|       1.0|
|       376067|       1.0|
|       376068|       0.0|
|       376070|       0.0|
|       376073|       1.0|
|       376074|       0.0|
|       376077|       1.0|
|       376086|       0.0|
|       376057|       1.0|
|       376087|       1.0|
|       376090|       0.0|
|       376092|       0.0|
|       376095|       0.0|
|       376100|       0.0|
|       376102|       1.0|
|       376107|       0.0|
|       376108|       0.0|
+-------------+----------+
only showing top 20 rows



In [22]:
predictions.show()

+-------------+--------------------+--------------------+--------------------+----------+
|customer_code|            features|       rawPrediction|         probability|prediction|
+-------------+--------------------+--------------------+--------------------+----------+
|       376061|(461,[0,5,124,125...|[1.52329811749461...|[0.30465962349891...|       1.0|
|       376063|(461,[0,5,123,125...|[1.39321444295713...|[0.27864288859142...|       0.0|
|       376065|(461,[0,5,123,125...|[0.96762188027107...|[0.19352437605421...|       0.0|
|       376066|(461,[0,5,123,125...|[1.60575448346845...|[0.32115089669366...|       1.0|
|       376067|(461,[0,59,124,12...|[1.52329811749461...|[0.30465962349891...|       1.0|
|       376068|(461,[0,5,123,125...|[1.45999872700422...|[0.29199974540084...|       0.0|
|       376070|(461,[0,5,124,125...|[1.00042979707796...|[0.20008595941559...|       0.0|
|       376073|(461,[0,5,123,125...|[1.60575448346845...|[0.32115089669366...|       1.0|
|       37

In [23]:
product_columns = [
    "savings_account", "guarantees", "current_accounts", "derivada_account", "payroll_account",
    "junior_account", "mas_particular_account", "particular_account", "particular_plus_account",
    "short_term_deposits", "medium_term_deposits", "long_term_deposits", "e_account", "funds",
    "mortgage", "loans", "taxes", "credit_card", "securities", "home_account", "payroll",
    "pensions_nom", "pension_plans", "direct_debit"
]

# Mapping dictionary: index to product column name
index_to_column = dict(enumerate(product_columns))

# Define a function to map prediction index to product column name
def map_prediction_to_column(row):
    customer_code = row.customer_code
    prediction_index = int(row.prediction)  # Convert prediction to integer index
    column_name = index_to_column.get(prediction_index, "unknown")  # Get corresponding column name
    return (customer_code, column_name)

# Apply mapping function to each row of predictions DataFrame
mapped_predictions_rdd = predictions.rdd.map(map_prediction_to_column)

# Convert RDD back to DataFrame with appropriate schema
mapped_predictions_df = spark.createDataFrame(mapped_predictions_rdd, ["customer_code", "predicted_product"])

# Show mapped predictions DataFrame

In [24]:
mapped_predictions_df.show()

+-------------+-----------------+
|customer_code|predicted_product|
+-------------+-----------------+
|       376061|       guarantees|
|       376063|  savings_account|
|       376065|  savings_account|
|       376066|       guarantees|
|       376067|       guarantees|
|       376068|  savings_account|
|       376070|  savings_account|
|       376073|       guarantees|
|       376074|  savings_account|
|       376077|       guarantees|
|       376086|  savings_account|
|       376057|       guarantees|
|       376087|       guarantees|
|       376090|  savings_account|
|       376092|  savings_account|
|       376095|  savings_account|
|       376100|  savings_account|
|       376102|       guarantees|
|       376107|  savings_account|
|       376108|  savings_account|
+-------------+-----------------+
only showing top 20 rows



In [27]:
from pyspark.sql.functions import count

mapped_predictions_df.groupBy("predicted_product").agg(count("*").alias("count")).show()

+--------------------+------+
|   predicted_product| count|
+--------------------+------+
|          guarantees|187250|
|             unknown|     1|
|     savings_account|740968|
|    current_accounts|   364|
|mas_particular_ac...|    38|
+--------------------+------+



In [29]:
187250+740968+364+38+1

928621

In [28]:
predictions.count()

928621

In [30]:
# Define Random Forest classifier
rf1 = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42, numTrees=10, maxDepth=5)  # Adjust parameters

In [31]:
# Fit pipeline on training data
model1 = rf1.fit(train_data)

In [32]:
# Make predictions on the transformed test data
predictions1 = model1.transform(test_data)

# Show predictions
predictions1.select("customer_code", "prediction").show()

+-------------+----------+
|customer_code|prediction|
+-------------+----------+
|       376061|       0.0|
|       376063|       0.0|
|       376065|       0.0|
|       376066|       1.0|
|       376067|       0.0|
|       376068|       0.0|
|       376070|       0.0|
|       376073|       1.0|
|       376074|       0.0|
|       376077|       1.0|
|       376086|       0.0|
|       376057|       1.0|
|       376087|       1.0|
|       376090|       0.0|
|       376092|       0.0|
|       376095|       0.0|
|       376100|       0.0|
|       376102|       0.0|
|       376107|       0.0|
|       376108|       0.0|
+-------------+----------+
only showing top 20 rows



In [33]:
predictions1.show()

+-------------+--------------------+--------------------+--------------------+----------+
|customer_code|            features|       rawPrediction|         probability|prediction|
+-------------+--------------------+--------------------+--------------------+----------+
|       376061|(461,[0,5,124,125...|[4.02950266888223...|[0.40295026688822...|       0.0|
|       376063|(461,[0,5,123,125...|[3.43820585236697...|[0.34382058523669...|       0.0|
|       376065|(461,[0,5,123,125...|[2.54086775251879...|[0.25408677525188...|       0.0|
|       376066|(461,[0,5,123,125...|[3.39912427007023...|[0.33991242700702...|       1.0|
|       376067|(461,[0,59,124,12...|[4.21092914134619...|[0.42109291413462...|       0.0|
|       376068|(461,[0,5,123,125...|[2.67313765624114...|[0.26731376562411...|       0.0|
|       376070|(461,[0,5,124,125...|[2.68787437272868...|[0.26878743727286...|       0.0|
|       376073|(461,[0,5,123,125...|[3.39912427007023...|[0.33991242700702...|       1.0|
|       37

In [34]:
product_columns = [
    "savings_account", "guarantees", "current_accounts", "derivada_account", "payroll_account",
    "junior_account", "mas_particular_account", "particular_account", "particular_plus_account",
    "short_term_deposits", "medium_term_deposits", "long_term_deposits", "e_account", "funds",
    "mortgage", "loans", "taxes", "credit_card", "securities", "home_account", "payroll",
    "pensions_nom", "pension_plans", "direct_debit"
]

# Mapping dictionary: index to product column name
index_to_column = dict(enumerate(product_columns))

# Define a function to map prediction index to product column name
def map_prediction_to_column(row):
    customer_code = row.customer_code
    prediction_index = int(row.prediction)  # Convert prediction to integer index
    column_name = index_to_column.get(prediction_index, "unknown")  # Get corresponding column name
    return (customer_code, column_name)

# Apply mapping function to each row of predictions DataFrame
mapped_predictions_rdd1 = predictions1.rdd.map(map_prediction_to_column)

# Convert RDD back to DataFrame with appropriate schema
mapped_predictions_df1 = spark.createDataFrame(mapped_predictions_rdd1, ["customer_code", "predicted_product"])

# Show mapped predictions DataFrame

In [35]:
mapped_predictions_df1.show()

+-------------+-----------------+
|customer_code|predicted_product|
+-------------+-----------------+
|       376061|  savings_account|
|       376063|  savings_account|
|       376065|  savings_account|
|       376066|       guarantees|
|       376067|  savings_account|
|       376068|  savings_account|
|       376070|  savings_account|
|       376073|       guarantees|
|       376074|  savings_account|
|       376077|       guarantees|
|       376086|  savings_account|
|       376057|       guarantees|
|       376087|       guarantees|
|       376090|  savings_account|
|       376092|  savings_account|
|       376095|  savings_account|
|       376100|  savings_account|
|       376102|  savings_account|
|       376107|  savings_account|
|       376108|  savings_account|
+-------------+-----------------+
only showing top 20 rows



In [37]:
from pyspark.sql.functions import count

mapped_predictions_df1.groupBy("predicted_product").agg(count("*").alias("count")).show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [38]:
# Define the local path where you want to save the model
model_local_path = "/content/model_rf3"

# Save the model to the local path
model1.save(model_local_path)

import shutil

# Define the destination directory in Google Drive
drive_destination = '/content/drive/My Drive/santander-product-recommendation/'

# Copy the model directory to Google Drive
shutil.copytree(model_local_path, drive_destination + 'model_rf3')

'/content/drive/My Drive/santander-product-recommendation/model_rf3'

In [39]:
# Define Random Forest classifier
rf3 = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42, numTrees=10, maxDepth=10)  # Adjust parameters

In [40]:
# Fit pipeline on training data
model3 = rf3.fit(train_data)

In [41]:
# Make predictions on the transformed test data
predictions3 = model3.transform(test_data)

# Show predictions
predictions3.select("customer_code", "prediction").show()

+-------------+----------+
|customer_code|prediction|
+-------------+----------+
|       376061|       1.0|
|       376063|       0.0|
|       376065|       0.0|
|       376066|       1.0|
|       376067|       1.0|
|       376068|       0.0|
|       376070|       0.0|
|       376073|       1.0|
|       376074|       1.0|
|       376077|       1.0|
|       376086|       0.0|
|       376057|       1.0|
|       376087|       1.0|
|       376090|       0.0|
|       376092|       0.0|
|       376095|       0.0|
|       376100|       0.0|
|       376102|       1.0|
|       376107|       0.0|
|       376108|       0.0|
+-------------+----------+
only showing top 20 rows



In [42]:
predictions3.show()

+-------------+--------------------+--------------------+--------------------+----------+
|customer_code|            features|       rawPrediction|         probability|prediction|
+-------------+--------------------+--------------------+--------------------+----------+
|       376061|(461,[0,5,124,125...|[3.49488623385267...|[0.34948862338525...|       1.0|
|       376063|(461,[0,5,123,125...|[2.69858340040924...|[0.26985834004092...|       0.0|
|       376065|(461,[0,5,123,125...|[1.99908453017820...|[0.19990845301781...|       0.0|
|       376066|(461,[0,5,123,125...|[3.10193780312427...|[0.31019378031244...|       1.0|
|       376067|(461,[0,59,124,12...|[3.72782585802832...|[0.37278258580281...|       1.0|
|       376068|(461,[0,5,123,125...|[2.07655485537414...|[0.20765548553741...|       0.0|
|       376070|(461,[0,5,124,125...|[2.12693304911208...|[0.21269330491120...|       0.0|
|       376073|(461,[0,5,123,125...|[3.05360539836273...|[0.30536053983628...|       1.0|
|       37

In [43]:
# Define the local path where you want to save the model
model_local_path = "/content/model_rf4"

# Save the model to the local path
model3.save(model_local_path)

import shutil

# Define the destination directory in Google Drive
drive_destination = '/content/drive/My Drive/santander-product-recommendation/'

# Copy the model directory to Google Drive
shutil.copytree(model_local_path, drive_destination + 'model_rf4')

'/content/drive/My Drive/santander-product-recommendation/model_rf4'

In [ ]:
from pyspark.sql.functions import lit

product_columns = [
    "savings_account", "guarantees", "current_accounts", "derivada_account", "payroll_account",
    "junior_account", "mas_particular_account", "particular_account", "particular_plus_account",
    "short_term_deposits", "medium_term_deposits", "long_term_deposits", "e_account", "funds",
    "mortgage", "loans", "taxes", "credit_card", "securities", "home_account", "payroll",
    "pensions_nom", "pension_plans", "direct_debit"
]
# Add missing product columns to test_data with default value 0
for col_name in product_columns:
    test_data = test_data.withColumn(col_name, lit(0))


In [ ]:
predictions = model.transform(test_data)

In [ ]:
# Step 5: Optionally, evaluate predictions or save results
predictions.select("prediction", "label", "features").show(5)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

In [ ]:
predictions = model.transform(test_data)

In [ ]:
# Example evaluation if you have labels in test data
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy on test data: {accuracy}")


In [ ]:
# Index the target column to convert string labels to numeric indices
target_indexer = StringIndexer(inputCol="multi_label", outputCol="label").fit(test_data)
test_data = target_indexer.transform(test_data)


In [ ]:
test_df = test_data.withColumn("multi_label", concat_ws('_', *product_columns))
test_df = indexer.transform(test_df)

# Transform the test data and make predictions
predictions = model.transform(test_df)

# Show predictions
predictions.select("customer_code", "prediction").show()# As